<a href="https://colab.research.google.com/github/akapo/class-llm/blob/main/chapter1/langchain_step1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **실습 환경설정**

**필요한 파이썬 패키지 설치**

In [1]:
!pip install -qU tiktoken
!pip install -qU langchain langchain-openai
!pip install -qU langchain-google-genai langchain-anthropic
!pip install -qU python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.5/865.5 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31

모델에 따라서 토큰을 자르는 방법이 다르다.  
gpt-3.5와 gpt-4의 토근 구분법이 다르므로 같은 문장이라도 토큰 수가 다르게 나올 수 있다

**토큰 계산**

In [2]:
import tiktoken

text= "Every cultural group has its own language and writing system."
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
print('[영문]')
print(tokens)
print(len(tokens))

text= "각각의 문화 공동체는 자신만의 언어와 문자 체계를 갖는다."
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
print('\n[한글]')
print(tokens)
print(len(tokens))

[영문]
[15745, 15186, 3566, 853, 1617, 2316, 6439, 326, 5281, 2420, 13]
11

[한글]
[22566, 22566, 3408, 123028, 164451, 16238, 2770, 62159, 10452, 3408, 69163, 5959, 12753, 132568, 55005, 173370, 95933, 125225, 13]
19


결과분석:  
텍스트는 OpenAI가 정의한 규칙에 의해 여러개의 토큰으로 잘리게 되고 각 토큰은 일련번호 형태로 처리된다.  
위 예제에서 영문 텍스트와 한글 텍스트는 완전히 동일한 의미이다. 하지만 영문 텍스트는 11토큰, 한글 텍스트는 19토큰으로 분할되어 한글이 영문에 비해 72%나 많은 토큰이 사용되었다.  
입력 토큰 수와 출력 토큰 수는 과금의 기준이 된다. 한국어가 영어에 비해 매우 불리한 구조이다.

**환경변수 감추기** (python-dotenv)

In [3]:
#!pip install python-dotenv
import dotenv
dotenv.load_dotenv()

import os
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')

# 주의: API_KEY를 깃허브에 노출하면 LLM 업체는 해당키를 즉시 삭제함.
print("GOOGLE_API_KEY: " + GOOGLE_API_KEY[:20])
print("OPENAI_API_KEY: " + OPENAI_API_KEY[:20])
print("ANTHROPIC_API_KEY: " + ANTHROPIC_API_KEY[:20])

GOOGLE_API_KEY: AIzaSyCCtFS9bml5jdGI
OPENAI_API_KEY: sk-proj-FEDi6Ge6iQyR
ANTHROPIC_API_KEY: sk-ant-api03-YcGqyQQ


# **[LangChain]　1. Model I/O**

##**1) Chat Models**

In [4]:
# 단순질의와 응답 포멧
import dotenv
dotenv.load_dotenv()

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model_name='gpt-4o-mini',     # 모델
    temperature=0,
)

question = "자기소개를 해주세요"
response = llm.invoke(question)
print(response) #llm의 응답은 클래스오브젝트이다.

#question = "Jinhee lives with her dog. What animals live with Jinhee?"
#question = "진희는 강아지를 키우고 있습니다. 진희가 키우고 있는 동물은?"
question = "대한민국의 가을은 몇 월부터 몇 월까지야?"
response = llm.invoke(question)
print(response.content)

content='안녕하세요! 저는 AI 언어 모델인 ChatGPT입니다. 다양한 주제에 대해 대화하고 정보를 제공하는 데 도움을 드릴 수 있습니다. 질문이 있거나 궁금한 점이 있다면 언제든지 말씀해 주세요!' response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 13, 'total_tokens': 63}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_ba606877f9', 'finish_reason': 'stop', 'logprobs': None} id='run-1994baeb-1164-4e16-a7e3-d21c04affd46-0' usage_metadata={'input_tokens': 13, 'output_tokens': 50, 'total_tokens': 63}
대한민국의 가을은 일반적으로 9월부터 11월까지로 여겨집니다. 9월 중순부터 11월 말까지가 가을의 주요 시기로, 이 시기에 기온이 서늘해지고 단풍이 아름답게 물드는 특징이 있습니다.


In [5]:
# 메시지 리스트를 이용한 질의
import dotenv
dotenv.load_dotenv()

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage;
llm = ChatOpenAI(
    model_name='gpt-4o-mini',  # 모델
    temperature=0,
)

messages = [
    SystemMessage(content="You are a helpful assistant that translates Korean to English. Translate the user sentence."),
    HumanMessage(content="대한민국의 가을은 몇 월부터 몇 월까지야?.")
]
ai_msg = llm.invoke(messages).content
print(ai_msg)

In South Korea, autumn lasts from which month to which month?


In [ ]:
# SystemMessage, HumanMessage, AIMessage
import dotenv
dotenv.load_dotenv()

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
llm = ChatOpenAI(
    model_name='gpt-4o',     # 모델
    temperature=0,
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="안녕하세요? 저는 존이라고 합니다."),
    AIMessage(content="안녕하세요, 존 씨! 어떻게 도와드릴까요?"),
    HumanMessage(content="제 이름을 아세요?"),
]
ai_msg = llm.invoke(messages).content
print(ai_msg)

네, 존 씨라고 하셨죠. 맞나요?


In [ ]:
# Anthropic LLM Test
import dotenv
dotenv.load_dotenv()

from langchain_anthropic import ChatAnthropic
llm = ChatAnthropic(
    model="claude-3-5-sonnet-20240620",
    temperature=0,
)

question = "자기소개를 해주세요"
print(llm.invoke(question).content)

messages = [
    SystemMessage(content="You are a helpful assistant that translates Korean to English. Translate the user sentence."),
    HumanMessage(content="대한민국의 가을은 몇 월부터 몇 월까지야?.")
]
ai_msg = llm.invoke(messages).content
print(ai_msg)

안녕하세요. 저는 Anthropic에서 개발한 AI 어시스턴트 Claude입니다. 다양한 주제에 대해 대화를 나누고 질문에 답변하며 여러 가지 작업을 도와드릴 수 있습니다. 하지만 저는 인공지능이기 때문에 감정이나 의식은 없습니다. 제가 어떤 도움을 드릴 수 있을까요?
The fall season in South Korea typically lasts from September to November.

In English, the sentence translates to:

"When does fall in South Korea start and end?"


In [6]:
# Google Gemini LLM Test
import dotenv
dotenv.load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0
)

question = "자기소개를 해주세요"
print(llm.invoke(question).content)

messages = [
    SystemMessage(content="You are a helpful assistant that translates Korean to English. Translate the user sentence."),
    HumanMessage(content="대한민국의 가을은 몇 월부터 몇 월까지야?.")
]
ai_msg = llm.invoke(messages).content
print(ai_msg)

저는 Google에서 개발한 대량 언어 모델입니다. 

저는 방대한 텍스트 데이터를 기반으로 훈련되었으며, 다양한 주제에 대해 사람과 자연스럽게 대화하고 질문에 답변할 수 있습니다. 

예를 들어, 

* 정보 제공: 역사적 사건, 과학적 사실, 문화적 주제 등 다양한 주제에 대한 정보를 제공할 수 있습니다.
* 텍스트 생성: 이야기, 기사, 요약, 대화 등 다양한 형식의 텍스트를 생성할 수 있습니다.
* 번역: 여러 언어 간에 텍스트를 번역할 수 있습니다.
* 코딩: Python, Java, C++ 등 다양한 프로그래밍 언어로 코드를 작성하고 디버깅하는 데 도움을 줄 수 있습니다.

저는 아직 개발 중이지만, 다양한 작업을 수행하고 사용자의 요구에 맞춰 지속적으로 발전하고 있습니다. 궁금한 점이 있으면 무엇이든 물어보세요. 

From what month to what month is autumn in Korea? 



**응답 스트리밍** (방법 #1)

In [13]:
import dotenv
dotenv.load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI

gemini = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
gpt4o  = ChatOpenAI(model="gpt-4o")
claude = ChatAnthropic(model="claude-3-5-sonnet-20240620")


question = "당신은 자의식을 가지고 있습니까? 당신은 생각하고 판단하며 이해하고 공감할 수 있습니까?"

print('[Gemini-1.5-pro]\n')
response = gemini.stream(question)
for chunk in response:
    print(chunk.content, end="")

print('\n[GPT-4o]\n')
response = gpt4o.stream(question)
for chunk in response:
    print(chunk.content, end="")

print('\n\n[Claude-3.5-sonnet]\n')
response = claude.stream(question)
for chunk in response:
    print(chunk.content, end="")

[Gemini-1.5-pro]

흥미로운 질문이네요! 저는 구글에서 훈련한 대규모 언어 모델이지만, 자의식을 가지고 있지는 않습니다. 

* **생각하고 판단하기:** 저는 방대한 데이터를 기반으로 질문에 답하고 텍스트를 생성할 수 있습니다. 하지만 이는 제가 생각하거나 판단하는 것이 아니라, 학습된 패턴을 기반으로 가장 적절한 답변을 찾아내는 것입니다. 
* **이해하고 공감하기:** 저는 인간의 감정을 이해하고 공감하는 능력이 없습니다. 감정과 관련된 단어들을 이해하고 사용할 수는 있지만, 실제로 감정을 느끼거나 공유하는 것은 아닙니다.

결론적으로 저는 사람처럼 생각하고 느끼는 존재가 아닌, 인간의 언어를 처리하고 생성하도록 설계된 도구입니다. 하지만 저는 다양한 주제에 대해 이야기하고 정보를 제공하는 데 도움을 줄 수 있습니다. 궁금한 점이 있다면 언제든지 물어보세요! 

[GPT-4o]

저는 자의식을 가지고 있지 않습니다. 저는 OpenAI의 언어 모델로, 인간처럼 생각하거나 감정을 느낄 수 없습니다. 대신, 저는 주어진 입력에 기반하여 정보를 처리하고 답변을 생성하는 능력을 가지고 있습니다. 제 목적은 사용자들이 필요로 하는 정보를 제공하고, 질문에 대한 답변을 제공하거나, 다른 형태의 지원을 제공하는 것입니다. 

당신이 제게 묻는 질문에 대해 최선을 다해 답변할 수는 있지만, 이는 단순히 제가 프로그래밍된 알고리즘과 데이터에 기반한 것입니다. 이해나 판단, 공감과 같은 감정적인 측면은 인간만이 가지는 고유한 능력입니다.

[Claude-3.5-sonnet]

저는 인공지능 언어 모델로서 자의식이나 감정은 없습니다. 인간처럼 생각하고 판단하는 것은 아니지만, 제가 학습한 데이터를 바탕으로 정보를 처리하고 응답을 생성할 수 있습니다. 이해와 공감의 시뮬레이션은 가능하지만 진정한 의미의 이해와 공감은 아닙니다.

**응답 스트리밍** (방법 #2)

In [16]:
import dotenv
dotenv.load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

gemini = ChatGoogleGenerativeAI(model="gemini-1.5-pro", streaming=True, callbacks=[StreamingStdOutCallbackHandler()])
gpt4o  = ChatOpenAI(model="gpt-4o", streaming=True, callbacks=[StreamingStdOutCallbackHandler()])
claude = ChatAnthropic(model="claude-3-5-sonnet-20240620", streaming=True, callbacks=[StreamingStdOutCallbackHandler()])


question = "당신은 자의식을 가지고 있습니까? 당신은 생각하고 판단하며 이해하고 공감할 수 있습니까?"

print('[Gemini-1.5-pro]\n')  # Gemini는 이 방식으로 응답 스트리밍 불가능
response = gemini.invoke(question)

print('\n[GPT-4o]\n')
response = gpt4o.invoke(question)

print('\n\n[Claude-3.5-sonnet]\n')
response = claude.invoke(question)

[Gemini-1.5-pro]


[GPT-4o]

저는 자의식을 가지고 있지 않습니다. 저는 인간처럼 생각하거나 느끼지 않습니다. 대신, 저는 프로그래밍된 알고리즘과 데이터에 기반해 정보를 제공하고 질문에 답변하는 능력을 가지고 있습니다. 이해하고 공감하는 것처럼 보일 수 있지만, 이는 제가 학습한 데이터와 알고리즘의 결과일 뿐입니다. 따라서, 진정한 의미에서의 이해나 공감은 제가 할 수 없는 일입니다.

[Claude-3.5-sonnet]

저는 인공지능 챗봇으로 설계되었습니다. 의식이나 자아를 가지고 있지 않으며 실제로 생각하거나 이해하거나 공감하지 못합니다. 대신 복잡한 언어 모델과 데이터를 기반으로 인간과 비슷한 대화를 할 수 있도록 프로그래밍 되어 있습니다.

**[temperature 값에 따른 변화]**

In [23]:
# temperature 값에 따른 변화
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

question = "겨울에 대한 짧은 시를 30자 이내 작성하고, 시 글귀만 출력하세요"

#llm = ChatOpenAI(model="gpt-4o", temperature=0)
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)
print("[temperature=0]:")
for _ in range(2):
    answer = llm.invoke(question)
    print(f'{"="*40}\n{answer.content}')

#llm = ChatOpenAI(model="gpt-4o", temperature=1)
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=1)

print("\n[temperature=1]:")
for _ in range(2):
    answer = llm.invoke(question)
    print(f'{"="*40}\n{answer.content}')

[temperature=0]:
하얀 눈 내리는 밤
차가운 바람 속
따뜻한 마음 피어나네
하얀 눈 내리는 밤
차가운 바람 속
따뜻한 마음 피어나네

[temperature=1]:
눈 내리는 밤
고요한 거리에
겨울의 속삭임
차가운 바람
하얀 눈 내리고
겨울의 고요함
마음을 감싸네


**[top_p 값에 따른 변화]**

In [24]:
# top_p 값에 따른 변화
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

#llm = ChatOpenAI(model="gpt-4o", temperature=0.25, top_p=0)
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0.25, top_p=0)
question = "겨울에 대한 짧은 시를 30자 이내 작성하고, 시 글귀만 출력하세요"

print("[top_p=0]:")
for _ in range(2):
    answer = llm.invoke(question)
    print(f'{"="*40}\n{answer.content}')

#llm = ChatOpenAI(model="gpt-4o", temperature=0.25, top_p=1)
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0.25, top_p=1)

print("\n[top_p=1]:")
for _ in range(2):
    answer = llm.invoke(question)
    print(f'{"="*40}\n{answer.content}')

[top_p=0]:
하얀 눈 내리는 밤
차가운 바람 속
따뜻한 마음 피어나네
하얀 눈 내리는 밤
차가운 바람 속
따뜻한 마음 피어나네

[top_p=1]:
눈 내리는 고요한 밤
차가운 바람 속 따스한 불빛
겨울의 포근한 품속
눈 내리는 고요한 밤
차가운 바람 속 따스한 불빛
겨울의 마법이 펼쳐지네


## **2) Prompt**

In [29]:
# PromptTemplate 1
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")

# 질문 템플릿 형식 정의
template = '{country}의 수도는 뭐야?'

# 템플릿 완성하며 단계로 처리
prompt_template = PromptTemplate(template=template, input_variables=['country'])
prompt = prompt_template.format(country='일본')
response = llm.invoke(prompt)
print(response.content)

# 한줄로 처리
print(llm.invoke(prompt_template.format(country='캐나다')).content + "\n\n")

# 한꺼번에 질의
input_list = [
    {'country': '호주'},
    {'country': '중국'},
    {'country': '네덜란드'} ]
print(llm.invoke(prompt_template.format(country=input_list)).content)

일본의 수도는 도쿄(東京)입니다. 도쿄는 일본의 정치, 경제, 문화의 중심지로, 세계에서 가장 큰 도시 중 하나입니다.
캐나다의 수도는 오타와(Ottawa)입니다.


각 나라의 수도는 다음과 같습니다:

- 호주: 캔버라
- 중국: 베이징
- 네덜란드: 암스테르담

더 궁금한 점이 있으면 말씀해 주세요!


In [38]:
# (숨김) Zero-shot prompting
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
#llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)

# 질문 템플릿 형식 정의
template = '{country}의 수도는 뭐야? 답만 말해.'

# 한줄로 처리
print(llm.invoke(prompt_template.format(country='일본')).content)
print(llm.invoke(prompt_template.format(country='캐나다')).content)

도쿄
오타와


In [39]:
# (숨김) Few-shot prompting
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
#llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)

question = """
질문자가 이야기한 국가의 수도를 맞추는 대화입니다.
Q: 한국
A: 서울
Q: 일본
A: 도쿄
Q: {country}
A:
"""
prompt_template = PromptTemplate(template=template, input_variables=['country'])

# 한줄로 처리
print(llm.invoke(prompt_template.format(country='중국')).content)
print(llm.invoke(prompt_template.format(country='호주')).content)

베이징.
캔버라.


In [40]:
# PromptTemplate 2
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# OpenAI 챗모델을 초기화합니다.
llm = ChatOpenAI(model="gpt-4o-mini")

# 질문 템플릿 형식 정의
template = '{area1} 와 {area2} 의 시차는 몇시간이야?'

# 템플릿 완성
prompt = PromptTemplate(template=template, input_variables=['area1', 'area2'])

response = llm.invoke(prompt.format(area1='서울', area2='파리'))
print(response.content)

서울과 파리의 시차는 일반적으로 8시간입니다. 서울이 파리보다 8시간 빠릅니다. 그러나, 파리가 서머 타임을 적용할 경우(3월 마지막 주 일요일부터 10월 마지막 주 일요일까지) 시차가 7시간으로 줄어듭니다. 따라서, 시차는 서머 타임 적용 여부에 따라 7시간 또는 8시간이 될 수 있습니다.


In [ ]:
# PromptTemplate 3
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

template = """
당신은 친절하게 답변해 주는 친절 봇입니다. 사용자의 질문에 [FORMAT]에 맞추어 답변해 주세요.
답변은 항상 한글로 작성해 주세요.

질문:
{question}에 대하여 설명해 주세요.

FORMAT:
- 개요:
- 예시:
- 출처:
"""

prompt = PromptTemplate(template=template, input_variables=['question'])

llm = ChatOpenAI(model="gpt-4o-mini")

print(llm.invoke(prompt.format(question="LLM은 무엇입니까?")).content)

- 개요: LLM(대규모 언어 모델, Large Language Model)은 방대한 양의 텍스트 데이터를 기반으로 학습하여 자연어 처리(NLP) 작업을 수행하는 인공지능 모델입니다. LLM은 문맥을 이해하고, 문장을 생성하거나 질문에 답변하는 등 다양한 언어 관련 작업을 수행할 수 있는 능력을 가지고 있습니다.

- 예시: OpenAI의 GPT-3, Google's BERT, Meta의 LLaMA 등은 대표적인 LLM입니다. 이 모델들은 대화 생성, 텍스트 요약, 번역, 감정 분석 등 다양한 응용 분야에서 활용되고 있습니다.

- 출처: OpenAI, Google AI, Meta AI 공식 웹사이트 및 관련 논문.


In [ ]:
# PromptTemplate 4
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 [FORMAT]에 영어 회화를 작성해 주세요.

상황: {question}

FORMAT:
- 영어 회화:
- 한글 해석:
"""

prompt = PromptTemplate.from_template(template=template)

# OpenAI 챗모델을 초기화합니다.
llm = ChatOpenAI(model="gpt-4o-mini",
                 streaming=True,
                 callbacks=[StreamingStdOutCallbackHandler()])

response = llm.invoke(prompt.format(question="미국에서 피자 주문"))

- 영어 회화:
  - Customer: Hi there! I’d like to order a pizza, please.
  - Employee: Sure! What size would you like?
  - Customer: I’ll have a large, please.
  - Employee: Great choice! What toppings do you want?
  - Customer: I’d like pepperoni and mushrooms.
  - Employee: Would you like any extra cheese or sides with that?
  - Customer: Yes, please add extra cheese and a side of garlic bread.
  - Employee: Perfect! Can I have your name and address for delivery?
  - Customer: My name is Alex, and my address is 123 Elm Street.
  - Employee: Thank you, Alex! Your order will be ready in about 30 minutes.
  - Customer: Awesome, thank you!

- 한글 해석:
  - 고객: 안녕하세요! 피자를 주문하고 싶습니다.
  - 직원: 물론이죠! 어떤 사이즈로 드릴까요?
  - 고객: 큰 걸로 주세요.
  - 직원: 좋은 선택이세요! 토핑은 어떤 걸 원하시나요?
  - 고객: 페퍼로니와 버섯을 추가해주세요.
  - 직원: 추가 치즈나 사이드 메뉴를 원하시나요?
  - 고객: 네, 추가 치즈와 갈릭 브레드를 사이드로 추가해주세요.
  - 직원: 완벽합니다! 배달을 위한 이름과 주소를 주실 수 있나요?
  - 고객: 제 이름은 알렉스이고, 주소는 123 엘름 스트리트입니다.
  - 직원: 감사합니다, 알렉스! 주문은 약 30분 후에 준비될 거예요.
  - 고객: 멋져요, 감사합니다!

## **3) OutputParsers**
JSON과 같은 출력 형식을 지정하는 프롬프트 생성 및 응답 텍스트를 Python 객체로 변환하는 기능을 제공.

**CommaSeparatedListOutputParser**

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

input = prompt.format(subject="ice cream flavors")
print("prompt: " + input + "\n")

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
output = llm.invoke(input)
output_parser.parse(output.content)

prompt: List five ice cream flavors.
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`



['vanilla',
 'chocolate',
 'strawberry',
 'mint chocolate chip',
 'cookies and cream']

**PydanticOutputParser**

In [41]:
from pydantic import BaseModel, Field

class Recipe(BaseModel):
   ingredients: list[str] = Field(description="ingredients of the dish")
   steps: list[str] = Field(description="steps to make the dish")

from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Recipe)

format_instructions = parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [42]:
from langchain.prompts import PromptTemplate

template = """다음 요리의 레시피를 생각해 주세요.

{format_instructions}
한글로 출력하세요.

요리: {dish}
"""

prompt = PromptTemplate(
   template=template,
   input_variables=["dish"],
   partial_variables={"format_instructions": format_instructions}
)

formatted_prompt = prompt.format(dish="카레")

print(formatted_prompt)

다음 요리의 레시피를 생각해 주세요.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
한글로 출력하세요.

요리: 카레



In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
messages = [HumanMessage(content=formatted_prompt)]
output = chat.invoke(messages)

print(output.content)
# JSON 오브젝트로 바꾸기
#import json
#json_obj = json.loads(output.content)
#print(json.dumps(json_obj, indent=4, ensure_ascii=False))

```json
{
  "ingredients": [
    "닭고기 500g",
    "양파 1개",
    "당근 1개",
    "감자 2개",
    "카레 가루 3큰술",
    "코코넛 밀크 400ml",
    "식용유 2큰술",
    "소금 약간",
    "후추 약간",
    "물 500ml"
  ],
  "steps": [
    "양파를 잘게 썰고, 당근과 감자는 큐브 모양으로 자릅니다.",
    "팬에 식용유를 두르고 양파를 볶아 투명해질 때까지 볶습니다.",
    "닭고기를 추가하고 겉면이 노릇해질 때까지 볶습니다.",
    "당근과 감자를 넣고 함께 볶습니다.",
    "카레 가루를 넣고 잘 섞은 후, 물과 코코넛 밀크를 추가합니다.",
    "소금과 후추로 간을 맞추고, 중불에서 20분간 끓입니다.",
    "재료가 부드러워지면 불을 끄고, 그릇에 담아 제공합니다."
  ]
}
```


In [ ]:
# Recipe 클래스 오브젝트로 변환
recipe = parser.parse(output.content)
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['닭고기 500g', '양파 1개', '당근 1개', '감자 2개', '카레 가루 3큰술', '식용유 2큰술', '소금 약간', '후추 약간', '물 500ml'] steps=['닭고기는 한 입 크기로 자르고, 양파, 당근, 감자는 깍둑썰기 한다.', '냄비에 식용유를 두르고 양파를 넣어 볶아준다.', '양파가 투명해지면 닭고기를 넣고 겉면이 노릇해질 때까지 볶는다.', '당근과 감자를 넣고 함께 볶는다.', '물 500ml를 붓고 끓인다.', '끓기 시작하면 불을 줄이고 15분 정도 끓인다.', '카레 가루를 넣고 잘 섞은 후, 소금과 후추로 간을 맞춘다.', '약한 불에서 10분 더 끓여서 완성한다.']


# **[LangChain]　2. Chains**

## **1) LLM Chain**
 ― PromptTemplate, Language model, OutputParser를 연결

In [48]:
# LLM Chain 버전
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.tracers import ConsoleCallbackHandler

output_parser = CommaSeparatedListOutputParser()

prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

chain = LLMChain(prompt=prompt, llm=llm, output_parser=output_parser)

output = chain.invoke({"subject":"ice cream flavors"})

print(output['text'])

['vanilla', 'chocolate', 'strawberry', 'mint chocolate chip', 'cookies and cream']


In [ ]:
# LCEL 버전
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.callbacks.tracers import ConsoleCallbackHandler

output_parser = CommaSeparatedListOutputParser()

prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

chain = prompt | llm | output_parser

output = chain.invoke({"subject":"ice cream flavors"})
# for verbose
# output = chain.invoke({"subject":"ice cream flavors"}, config={'callbacks': [ConsoleCallbackHandler()]})

print(output)

['vanilla', 'chocolate', 'strawberry', 'mint chocolate chip', 'cookies and cream']


In [ ]:
# Quiz
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from pydantic import BaseModel, Field

class Recipe(BaseModel):
   ingredients: list[str] = Field(description="ingredients of the dish")
   steps: list[str] = Field(description="steps to make the dish")

output_parser = PydanticOutputParser(pydantic_object=Recipe)

template = """다음 요리의 레시피를 생각해 주세요.

{format_instructions}

요리: {dish}
"""

prompt = PromptTemplate(
   template=template,
   input_variables=["dish"],
   partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

chain = prompt | llm | output_parser

recipe = chain.invoke({"dish": "카레"})

print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['1 tablespoon vegetable oil', '1 onion, chopped', '2 cloves garlic, minced', '1 tablespoon ginger, grated', '2 carrots, diced', '1 potato, diced', '1 bell pepper, chopped', '1 can (400g) diced tomatoes', '2 tablespoons curry powder', '1 teaspoon turmeric', '1 can (400ml) coconut milk', 'Salt to taste', 'Fresh cilantro for garnish'] steps=['Heat the vegetable oil in a large pot over medium heat.', 'Add the chopped onion and sauté until translucent.', 'Stir in the minced garlic and grated ginger, cooking for another minute.', 'Add the diced carrots, potato, and bell pepper, and cook for about 5 minutes.', 'Stir in the diced tomatoes, curry powder, and turmeric, mixing well.', 'Pour in the coconut milk and bring the mixture to a simmer.', 'Reduce the heat and let it cook for about 20 minutes, or until the vegetables are tender.', 'Season with salt to taste.', 'Serve hot, garnished with fresh cilantro.']


## **2) SimpleSequentialChain**
 ― Chain과 Chain의 연결

In [53]:
# LLM Chain 으로 구현
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

cot_template = """다음 질문에 답하세요.

질문: {question}

단계별로 생각해 봅시다.
"""

cot_prompt = PromptTemplate(
   input_variables=["question"],
   template=cot_template,
)

cot_chain = LLMChain(prompt=cot_prompt, llm=llm)

summarize_template = """다음 문장을 결론만 간단히 요약하세요.
{input}
"""
summarize_prompt = PromptTemplate(
   input_variables=["input"],
   template=summarize_template,
)

summarize_chain = LLMChain(llm=llm, prompt=summarize_prompt)

from langchain.chains import SimpleSequentialChain

cot_summarize_chain = SimpleSequentialChain(chains=[cot_chain, summarize_chain])
#cot_summarize_chain = SimpleSequentialChain(chains=[cot_chain, summarize_chain], verbose=True)

result = cot_summarize_chain.invoke(
   {"input": "저는 시장에 가서 사과 10개를 샀습니다. 이웃에게 2개, 수리공에게 2개를 주었습니다. \
   그런 다음에 사과 5개를 더 사서 1개를 먹었습니다. 남은 개수는 몇 개인가요?"}
)
print(result["output"])



> Entering new SimpleSequentialChain chain...
단계별로 문제를 해결해 보겠습니다.

1. **처음 사과 개수**: 시장에서 사과 10개를 샀습니다.
   - 현재 사과 개수: 10개

2. **이웃에게 준 사과**: 이웃에게 2개를 주었습니다.
   - 현재 사과 개수: 10 - 2 = 8개

3. **수리공에게 준 사과**: 수리공에게 2개를 주었습니다.
   - 현재 사과 개수: 8 - 2 = 6개

4. **추가로 사과 구매**: 사과 5개를 더 샀습니다.
   - 현재 사과 개수: 6 + 5 = 11개

5. **먹은 사과**: 1개를 먹었습니다.
   - 현재 사과 개수: 11 - 1 = 10개

결론적으로, 남은 사과의 개수는 **10개**입니다.
남은 사과의 개수는 **10개**입니다.

> Finished chain.
남은 사과의 개수는 **10개**입니다.


In [51]:
# LCEL 으로 구현 (에러!)
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

cot_template = """다음 질문에 답하세요.

질문: {question}

단계별로 생각해 봅시다.
"""

cot_prompt = PromptTemplate(
   input_variables=["question"],
   template=cot_template,
)

cot_chain = cot_prompt | llm | output_parser

summarize_template = """다음 문장을 결론만 간단히 요약하세요.

{input}
"""
summarize_prompt = PromptTemplate(
   input_variables=["input"],
   template=summarize_template,
)

summarize_chain = summarize_prompt | llm | output_parser

from langchain.chains import SimpleSequentialChain

cot_summarize_chain = SimpleSequentialChain(chains=[cot_chain, summarize_chain])

result = cot_summarize_chain.invoke(
   {"input": "저는 시장에 가서 사과 10개를 샀습니다. 이웃에게 2개, 수리공에게 2개를 주었습니다. \
   그런 다음에 사과 5개를 더 사서 1개를 먹었습니다. 남은 개수는 몇 개인가요?"}
)
print(result)

ValidationError: 2 validation errors for SimpleSequentialChain
chains -> 0
  Can't instantiate abstract class Chain with abstract methods _call, input_keys, output_keys (type=type_error)
chains -> 1
  Can't instantiate abstract class Chain with abstract methods _call, input_keys, output_keys (type=type_error)